## Sylvester.Arithmetic

### Introduction
This [library](https://github.com/allisterb/Sylvester/tree/master/src/Base/Sylvester.FixedPoint) implements [lightweight](http://okmij.org/ftp/Computation/lightweight-dependent-typing.html) dependently typed natural number arithmetic and constraints in F# which enable arithmetic operations like bounds checking to be performed at compile-time by the F# type checker as long as the values for the operations are known at compile-time. 

In [2]:
/// Use the Sylvester.Arithmetic NuGet package in this notebook// Use the Sylvester arithmetic and collection packages
# r "nuget:Sylvester.Arithmetic"

open Sylvester.Arithmetic
open Sylvester.Arithmetic.N10

///Create typed representations of some natural numbers
let a,b,c = new n<400>(), new n<231111>(), new n<6577700>()

a + b + c

Val,Digits,Digit1,Digit2,Digit3,Digit4,Digit5,Digit6,Digit7,Digit8,Digit9,Digit10
6809211,"{ System.Tuple<Sylvester.Arithmetic.Base10+0,Sylvester.Arithmetic.Base10+0,Sylvester.Arithmetic.Base10+0,Sylvester.Arithmetic.Base10+6,Sylvester.Arithmetic.Base10+8,Sylvester.Arithmetic.Base10+0,Sylvester.Arithmetic.Base10+9,System.Tuple<Sylvester.Arithmetic.Base10+2,Sylvester.Arithmetic.Base10+1,Sylvester.Arithmetic.Base10+1>>: Item1: { Sylvester.Arithmetic.Base10+0: }, Item2: { Sylvester.Arithmetic.Base10+0: }, Item3: { Sylvester.Arithmetic.Base10+0: }, Item4: { Sylvester.Arithmetic.Base10+6: }, Item5: { Sylvester.Arithmetic.Base10+8: }, Item6: { Sylvester.Arithmetic.Base10+0: }, Item7: { Sylvester.Arithmetic.Base10+9: }, Rest: { System.Tuple<Sylvester.Arithmetic.Base10+2,Sylvester.Arithmetic.Base10+1,Sylvester.Arithmetic.Base10+1>: Item1: { Sylvester.Arithmetic.Base10+2: }, Item2: { Sylvester.Arithmetic.Base10+1: }, Item3: { Sylvester.Arithmetic.Base10+1: } } }",{ Sylvester.Arithmetic.Base10+1: },{ Sylvester.Arithmetic.Base10+1: },{ Sylvester.Arithmetic.Base10+2: },{ Sylvester.Arithmetic.Base10+9: },{ Sylvester.Arithmetic.Base10+0: },{ Sylvester.Arithmetic.Base10+8: },{ Sylvester.Arithmetic.Base10+6: },{ Sylvester.Arithmetic.Base10+0: },{ Sylvester.Arithmetic.Base10+0: },{ Sylvester.Arithmetic.Base10+0: }


These values have types derived from *Sylvester.Arithmetic.N10*. The type of ``a`` is *N10<0,0,0,0,0,0,0,4,0,0>* and the type of ``c`` is *N10<0,0,0,6,5,7,7,7,0,0>*.

In [14]:
c.GetType()

Sylvester.Arithmetic.N10+N10`10[Sylvester.Arithmetic.Base10+0,Sylvester.Arithmetic.Base10+0,Sylvester.Arithmetic.Base10+0,Sylvester.Arithmetic.Base10+6,Sylvester.Arithmetic.Base10+5,Sylvester.Arithmetic.Base10+7,Sylvester.Arithmetic.Base10+7,Sylvester.Arithmetic.Base10+7,Sylvester.Arithmetic.Base10+0,Sylvester.Arithmetic.Base10+0]

The types of the results of arithmetic operations depend on the values of the operands.

In [3]:
let d = (a + b + c) * four

d

N<27236844UL>

In [15]:
d.GetType()

Sylvester.Arithmetic.N10+N10`10[Sylvester.Arithmetic.Base10+0,Sylvester.Arithmetic.Base10+0,Sylvester.Arithmetic.Base10+2,Sylvester.Arithmetic.Base10+7,Sylvester.Arithmetic.Base10+2,Sylvester.Arithmetic.Base10+3,Sylvester.Arithmetic.Base10+6,Sylvester.Arithmetic.Base10+8,Sylvester.Arithmetic.Base10+4,Sylvester.Arithmetic.Base10+4]

This enables type-level constraints to be written which run at compile-time

In [26]:
N<4>.i +== four ///Compare two instances of N<4>

Sylvester.Arithmetic.Bool+True

In [27]:
///Type level comparison operators return types dependent on the result of the comparison
N<4>.i +== thousand

Sylvester.Arithmetic.Bool+False

In [28]:
/// The check functions performs type level checks. 
check(d +< ten) ///This check causes a type error at compile time

Type mismatch. Expecting a
    'Constraint<Success>'    
but given a
    'Constraint<Failure>'    
The type 'Success' does not match the type 'Failure'

In [18]:
a - (two * a) /// A negative number results in a type representing an underflow at compile-time 

N10Underflow

In [19]:
/// Define a function myop using type-level arithmetic constraintss
let myop a b c d =
    check(a +> b) /// Use the +> and +< operators for type-level comparisons
    check (b +== zero) /// +== Type level equality operator
    check (c +== (a + one))
    a + b + c + d
    
/// Call the function with values that don't satisfy the constraints 
/// This will cause a type error and the code won't compile
myop four N<1000>.i three two 

Type mismatch. Expecting a
    'Constraint<Success>'    
but given a
    'Constraint<Failure>'    
The type 'Success' does not match the type 'Failure'
Type mismatch. Expecting a
    'Constraint<Success>'    
but given a
    'Constraint<Failure>'    
The type 'Success' does not match the type 'Failure'

In [22]:
let myop a b c d =
    check(a +> b)
    check (b +== zero)
    check (c +== (a + one))
    a + b + c + d
    
/// This call to myop satisfies the arithmetic constraints so the code compiles
myop five zero six N<10>.i

N<21UL>

Mixing natural number and primitive types result in the natural numbers being lowered to their primitive representation.

In [3]:
seven + eight

N<15UL>

In [4]:
seven + eight + 1


16

In IDEs with IntelliSense like Visual Studio you can get immediate feedback on type checks
as you enter code:

![SylvesterIDE2.png](SylvesterIDE2.png)

### How it works
Natural numbers are represented by fixed=point decimal types. The basic concept is described here: https://wiki.haskell.org/The_Monad.Reader/Issue5/Number_Param_Types#Fixed-precision_decimal_types

and an initial implementation for F# was created by MingTang here: https://github.com/Ming-Tang/TypeLevel/blob/master/TypeLevel/Base10.fs

The implementation in Sylvester.Arithmetic uses a 10-digit fixed-point representation as well as adding the two arithmetic comparison operators +< and +>. It also utilizes the F# type provider mechanism to make creating types out of compile-time numeric values as seamless as possible.